# NET ART REVOLUTION?

**Abstract**

The advent of the internet and personal computers as tools of communication and creation radically transformed the world we live in over the past 40 years. The impacts of both technologies across various fields and topics have been well documented, including their impact on the art world. Regarding the latter, discussion is often focused on how the internet and computers have transformed art creation as well as the ways in which memory institutions function and interact with their audiences. Slightly less focus has been given on how the technologies may have impacted how artworks are acquired. Net Art Revolution? aims to look at this particular aspect of the digital revolution by analyzing data from two different NYC-based institutions -- MoMA, representing the more traditional art world, and Rhizome, a smaller institution dedicated to digital-born art and culture, often refered to as net art. Both institutions make data about their collections publicly available -- MoMA in csv and json formats, Rhizome as Linked Open Data -- and this data is used to investigate our primary research question: **Did the internet and personal computers have an impact on the make up of art collections?** 


This project was undertaken by Margherita Donelli, Chiara Catizone, and Laurent Fintoni for the Electronic Publishing and Digital Storytelling (a.y. 2021-2022) final examination, as part of the *Digital Humanities and Digital Knowledge Master's Degree* at the University of Bologna.

***

In [1]:
#IMPORTS + PATH
path = './'

#GENERIC
import sys
import pandas as pd
import numpy as np
import re
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
#PLOTLY
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pycountry
import chart_studio.tools as tls
import ipywidgets as ipw
from statsmodels.graphics.mosaicplot import mosaic
from plotly.subplots import make_subplots
#PREPPING/CLEANING
from SPARQLWrapper import SPARQLWrapper, JSON
from functools import reduce
from collections import defaultdict
pio.templates.default = "simple_white"
#SCRAPING
from __future__ import print_function
from bs4 import BeautifulSoup
import requests
#KEYWORDS
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize 
from nltk.util import ngrams

In [59]:
#FUNCTIONS

#CLEAN DATA
def normalizeDate(year):
    if  re.match('.*\d\d\d\d.*', str(year)):
        newDate = re.findall(r'\d\d\d\d', year)
        #modify the date in order to normalize it using the first year in the list 
        return int(newDate[0])
        
    else: 
        return 0

def getGender(ids):
    listIds = str(ids).split(', ')
    listGenders = list()
    for id in listIds:
        gender = artists[artists['ID']== id]
        if len(gender)>0:
            gender = gender['Gender'].values[0]
            listGenders.append(gender)
        else:
            listGenders.append('missing')

    
    return ", ".join(listGenders)

def getNationality(ids):
    listIds = str(ids).split(', ')
    listNationality = list()
    for id in listIds:
        nationality = artists[artists['ID']== id]
        if len(nationality)>0:
            nationality = nationality['Nationality'].values[0]
            listNationality.append(nationality)
        else:
            listNationality.append('missing')

    
    return ", ".join(listNationality)

#SPARQL QUERY
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

#Scrape summaries, descriptions and artists statements from Rhizome website 
def url_to_text_rhizome(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, "html.parser")
    accordion = [p.text.strip() for p in soup.find(id="AccordionDescriptionBody").find_all('div')]
    print(url)
    return accordion

#Extract text from MoMA website w/ exclusions for 404 status, missing URLs, and pages w/ no despcription
def url_to_text_moma(url):
    if url != 'missing':
        page = requests.get(url)
        status = page.status_code
        if status != 404:
            soup = BeautifulSoup(page.content, "html.parser")
            try:
                if soup.find(class_="uneven-columns--work").find(class_='main-content') is not None:
                    text = soup.find(class_="uneven-columns--work").find(class_='main-content').find_all('p')
                else:
                    text = ''
            except AttributeError:
                text = ''
                pass
        else:
            text = '404'
    else:
        text = 'missing'     
    print(url)
    return text

#Extract keywords Rhizome and pass them to a new column
def get_keywords(row):
    punct_tokenizer = nltk.RegexpTokenizer(r"\w+")
    some_text = row['Text']
    tokens = punct_tokenizer.tokenize(some_text)
    keywords = [keyword for keyword in tokens if keyword.isalpha() and not keyword in stop_words]
    freqdist = nltk.FreqDist(keywords)
    most_common = freqdist.most_common(20)
    most_common_list = []
    for t in most_common:
        most_common_list.append(t[0])
    keywords_string = ', '.join(most_common_list)
    return keywords_string

#get keywords MoMA 
def get_keywords_2(row):
    punct_tokenizer = nltk.RegexpTokenizer(r"\w+")
    some_text = row['text']
    tokens = punct_tokenizer.tokenize(some_text)
    keywords = [keyword for keyword in tokens if keyword.isalpha() and not keyword in stop_words]
    freqdist = nltk.FreqDist(keywords)
    most_common = freqdist.most_common(20)
    most_common_list = []
    for t in most_common:
        most_common_list.append(t[0])
    keywords_string = ', '.join(most_common_list)
    return keywords_string

#group genders 
def group_genders(df):
    collectives = ['M', 'F', 'missing', 'NB']
    df = df['Gender'].value_counts().rename_axis('Gender').reset_index(name='Counts')
    df_collectives = ['Collectives', df[df.Gender.str.contains(',')]['Counts'].sum()]
    df = df.loc[~df.Gender.isin(collectives) == False].reset_index(drop=True)
    df.loc[5] = df_collectives
    df = df.sort_values(by='Counts', ascending=False).replace('M', 'Male').replace('F', 'Female').replace('missing', 'Missing')
    return df

#Turn nationalities into countries
def getContinent(string):
    if string in north:
        return 'North Globe'
    elif string in south:
        return 'South Globe'
    else: 
        'missing'

#Be fuzzy!
def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
        return result[0].alpha_3
    except:
        return np.nan

In [3]:
#LOAD ALL PICKLED DATA

#complete DFs
rhz_artworks = pd.read_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra.pkl')
rhz_artworks = rhz_artworks.replace('N/B', 'NB')
moma_artworks = pd.read_pickle(path+'MOMA_data/pickle/MoMAartworks.pkl')
moma_artworks_old =  pd.read_pickle(path+'MOMA_data/pickle/old_artworks.pkl')
moma_artworks_new = pd.read_pickle(path+'MOMA_data/pickle/new_artworks.pkl')
moma_rhz_compare = moma_artworks_new.loc[moma_artworks_new['dateAcquired'] >= 2000]
moma_rhz_compare = moma_rhz_compare.loc[moma_rhz_compare['dateCreated'] >= 1983]

#MoMA department DFs
moma_draw = moma_artworks.loc[moma_artworks['Department'] == 'Drawings & Prints']
moma_draw_before = moma_draw.loc[moma_draw['dateCreated'] <= 1982]
moma_draw_after = moma_draw.loc[moma_draw['dateCreated'] >= 1983]
moma_films = moma_artworks.loc[moma_artworks['Department'] == 'Film']
moma_films_before = moma_films.loc[moma_films['dateCreated'] <= 1982]
moma_films_after = moma_films.loc[moma_films['dateCreated'] >= 1983]
moma_media = moma_artworks.loc[moma_artworks['Department'] == 'Media and Performance']
moma_media_before = moma_media.loc[moma_media['dateCreated'] <= 1982]
moma_media_after = moma_media.loc[moma_media['dateCreated'] >= 1983]
moma_photo = moma_artworks.loc[moma_artworks['Department'] == 'Photography']
moma_photo_before = moma_photo.loc[moma_photo['dateCreated'] <= 1982]
moma_photo_after = moma_photo.loc[moma_photo['dateCreated'] >= 1983]

#Rhizome with text
rhizome_txt_clean = pd.read_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra_text_clean.pkl')
rhizome_txt_stop_kw = pd.read_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra_text_clean_stop_keywords.pkl')

#MoMA with text
moma_arch_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text_final_stop.pkl')
moma_arch_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only_final_stop.pkl')
moma_draw_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text_final_stop.pkl')
moma_draw_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text_final_stop.pkl')
moma_films_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/films_cont_text_final_stop.pkl')
moma_films_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/films_mod_text_final_stop.pkl')
moma_media_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text_final_stop.pkl')
moma_media_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text_final_stop.pkl')
moma_paint_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text_final_stop.pkl')
moma_paint_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text_final_stop.pkl')
moma_photo_cont_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_cont_text_final_stop.pkl')
moma_photo_mod_text_stop = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_mod_text_final_stop.pkl')
moma_full_text_stop = pd.concat([moma_arch_cont_text_stop, moma_draw_cont_text_stop, moma_draw_mod_text_stop, moma_films_cont_text_stop, moma_films_mod_text_stop, moma_media_cont_text_stop, moma_media_mod_text_stop, moma_paint_cont_text_stop, moma_paint_mod_text_stop, moma_photo_cont_text_stop, moma_photo_mod_text_stop])


***

## Assessing the Available Data (Everyone)

Our research began by looking at the available datasets. We downloaded [MoMA's publicly available dumps](https://github.com/MuseumofModernArt/collection) and looked at their formatting, and we then explored Rhizome's dataset (AKA the ArtBase) via their [SPARQL endpoint](https://query.artbase.rhizome.org/). The first key observations were as follows:
- MoMA's artworks and artists datasets are linked via an ID system, which did not exist in Rhizome
- Rhizome's dataset has more missing information such as biographical information of artists 
- The sizes of the datasets are drastically different with Rhizome's artists set being 10% of MoMA's and its artworks set being 1.6% percent of MoMA's 

We contacted Rhizome with some questions about how their data was formatted and they did eventually supply us with a copy of their RDF dump as of February 2022. We used this RDF file to primarily check our queries and extracted all the data we needed directly via the endpoint. We would like to acknowledge and thank Dragan Espenschied, Preservation Director at Rhizome, for his willingness to answer some of our questions about the formatting of Rhizome's data and supplying us with a copy of the RDF dump. 

***

## Cleaning and Preparing MoMA Data (Chiara)

We first began by spliting the MoMA artworks dataset using 1983 as a pivot date for date created, as this is the year that the first artwork was created in Rhizome as well as the year in which TCP/IP was standardized, in effect a turning point towards the ubiquity of the worldwide web. This process removed circa 3000 entries that didn't have creation date.<br><br>
We then selected the columns most relevant to our research, as well as most aligned with existing Rhizome data formatting. These were: Title; Artist; ConstituentID (linking system); Date (renamed to Date Created); Medium; Department; DateAcquired; URL (for web scraping); ThumbnailURL (for potential use of images). We also decided to split the artworks dataset by Departments, created subsequent filtered sets for potential more meaningful analysis with Rhizome's datasets.<br><br>
For the artists dataset all columns were kept as they were deemed potentially useful and we then used the Constituent ID mechanism to populate the artworks dataset with two key values for our analysis: gender and nationality. This way we would be able to work primarily from the artworks dataset. <br><br>
Finally, considering that we knew there would be missing data relevant to our research (such as gender) we decided to normalize all missing categorical values using the string 'missing' and all numerical values using 0 and transformed all dates to 4-digit years as we did not want to go more granular than the year level when looking at time. 

In [ ]:
#select and clean artist columns 
original_artists = pd.read_csv(path+'MOMA_data/Artists.csv')
artists= original_artists[['ConstituentID','DisplayName','Nationality','Gender','BeginDate','EndDate','Wiki QID','ULAN']]
artists.rename(columns={'ConstituentID': 'ID', 'DisplayName': 'Artist','BeginDate': 'Birth', 'EndDate': 'Death'}, inplace=True)
artists['Gender'] = artists['Gender'].fillna('missing').astype('str')
artists['Wiki QID'] = artists['Wiki QID'].fillna('missing').astype('str')
artists['ULAN'] = artists['ULAN'].fillna('0').astype('int')
artists['ID'] = artists['ID'].fillna('0').astype('str')
artists['Artist'] = artists['Artist'].fillna('Unknown').astype('str')
artists['Nationality'] = artists['Nationality'].fillna('missing').astype('str')
artists['Gender'] = artists['Gender'].fillna('missing').astype('str')
artists['Birth'] = artists['Birth'].fillna('0').astype('int')
artists['Death'] = artists['Death'].fillna('0').astype('int')

#normalize gender labels to M, F, NB for ease of comparison w/ Rhizome
for x, row in artists.iterrows():
    if re.match(r'[M|m]ale', row.Gender):
        artists.at[x, 'Gender'] = 'M'
    elif re.match(r'[F|f]emale',row.Gender):
        artists.at[x, 'Gender'] = 'F'
    elif re.match(r'Non-[B|b]inary', row.Gender):
        artists.at[x, 'Gender'] = 'NB'

#clean original artworks df
original_artworks = pd.read_csv(path+'Artworks.csv')
original_artworks =  original_artworks[['Title','Artist','ConstituentID','Date','Medium','Department','DateAcquired','URL','ThumbnailURL']]
original_artworks['Date'] = original_artworks['Date'].apply(lambda x: normalizeDate(x))
original_artworks['DateAcquired']=original_artworks['DateAcquired'].where((original_artworks['DateAcquired'].str.len() <= 4), original_artworks['DateAcquired'].str[0:4])
original_artworks['Gender'] = original_artworks['ConstituentID'].apply(lambda x: getGender(x))
original_artworks['Nationality'] = original_artworks['ConstituentID'].apply(lambda x: getNationality(x))

original_artworks = original_artworks.rename(columns={"Date": "dateCreated", "DateAcquired": "dateAcquired"})
original_artworks['Title'] = original_artworks['Title'].fillna('Unknown')
original_artworks['DateAcquired'] = original_artworks['DateAcquired'].astype('str')
original_artworks['DateAcquired'] = original_artworks['DateAcquired'].replace('nan', str('0'))
original_artworks['DateAcquired']=original_artworks['DateAcquired'].astype('int')

#pickle for reuse
#artists.to_pickle(path+'MoMA_data/pickle/MoMAartists.pkl')
#original_artworks.to_pickle(path+'MoMA_data/pickle/MoMAartworks.pkl')

#create time-based subsets
before83 = original_artworks[original_artworks.dateCreated < 1983]
after83 = original_artworks[original_artworks.dateCreated >= 1983]
#before83.to_pickle(path+'MoMA_data/pickle/old_artworks.pkl')
#after83.to_pickle(path+'MoMA_data/pickle/new_artworks.pkl')

#split artwork datasets further based on department for ease of comparison
architecture_design = before83[before83['Department'] == "Architecture & Design"]
#architecture_design.to_pickle(f'../MoMA_data/pickle/departments/architecture_design_mod.pkl')
architecture_design_img  = before83[before83['Department'] == "Architecture & Design - Image Archive"]
#architecture_design_img.to_pickle(f'../MoMA_data/pickle/departments/architecture_design_img_mod.pkl')
draws_prints= before83[before83['Department'] == "Drawings & Prints"]
#draws_prints.to_pickle(f'../MoMA_data/pickle/departments/draws_prints_mod.pkl')
films= before83[before83['Department'] == "Film"]
#films.to_pickle(f'../MoMA_data/pickle/departments/films_mod.pkl')
fluxus= before83[before83['Department'] == "Fluxus Collection"]
#fluxus.to_pickle(f'../MoMA_data/pickle/departments/fluxus_mod.pkl')
media_perf= before83[before83['Department'] == "Media and Performance"]
#media_perf.to_pickle(f'../MoMA_data/pickle/departments/media_perf_mod.pkl')
painting_sculp= before83[before83['Department'] == "Painting & Sculpture"]
painting_sculp.to_pickle(f'../MoMA_data/pickle/departments/paint_sculp_mod.pkl')
photo= before83[before83['Department'] == "Photography"]
#photo.to_pickle(f'../MoMA_data/pickle/departments/photo_mod.pkl')
architecture_design = after83[after83['Department'] == "Architecture & Design"]
architecture_design.to_pickle(f'../MoMA_data/pickle/departments/architecture_design_cont.pkl')
architecture_design_img  = after83[after83['Department'] == "Architecture & Design - Image Archive"]
architecture_design_img.to_pickle(f'../MoMA_data/pickle/departments/architecture_design_img_cont.pkl')
draws_prints= after83[after83['Department'] == "Drawings & Prints"]
#draws_prints.to_pickle(f'../MoMA_data/pickle/departments/draws_prints_cont.pkl')
films= after83[after83['Department'] == "Film"]
#films.to_pickle(f'../MoMA_data/pickle/departments/films_cont.pkl')
fluxus= after83[after83['Department'] == "Fluxus Collection"]
#fluxus.to_pickle(f'../MoMA_data/pickle/departments/fluxus_cont.pkl')
media_perf= after83[after83['Department'] == "Media and Performance"]
#media_perf.to_pickle(f'../MoMA_data/pickle/departments/media_perf_cont.pkl')
painting_sculp= after83[after83['Department'] == "Painting & Sculpture"]
#painting_sculp.to_pickle(f'../MoMA_data/pickle/departments/paint_sculp_cont.pkl')
photo= after83[after83['Department'] == "Photography"]
#photo.to_pickle(f'../MoMA_data/pickle/departments/photo_cont.pkl')


In [10]:
#show result
original_artworks = pd.read_pickle(path+'MoMA_data/pickle/MoMAartworks.pkl')
original_artworks.head()

,Title,Artist,ConstituentID,dateCreated,Medium,Department,dateAcquired,URL,ThumbnailURL,Gender,Nationality
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,1896,Ink and cut-and-pasted painted pages on paper,Architecture & Design,1996,http://www.moma.org/collection/works/2,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,M,Austrian
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,1987,Paint and colored pencil on print,Architecture & Design,1995,http://www.moma.org/collection/works/3,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,M,French
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,1903,"Graphite, pen, color pencil, ink, and gouache ...",Architecture & Design,1997,http://www.moma.org/collection/works/4,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,M,Austrian
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Photographic reproduction with colored synthet...,Architecture & Design,1995,http://www.moma.org/collection/works/5,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,M,missing
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,1903,"Graphite, color pencil, ink, and gouache on tr...",Architecture & Design,1997,http://www.moma.org/collection/works/6,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,M,Austrian


***

## Cleaning and Preparing Rhizome Data (Laurent & Margherita)

We began our exploration of Rhizome's dataset with a variety of SPARQL queries before settling on the following three to help us build artworks and artists datasets that could be equivalent to MoMA's: all the artists with available biographic information, all artworks with available metadata (title, dates, associated artist, website URLs), and artists who are/aren't part of a collective. These queries were conducted directly on the endpoint (and checked against the local RDF file for consistency) to retrieve them as csv files and are replicated below formatted using the SPARQLwrapper library.


In [ ]:
#set endpoint

endpoint_url = "https://query.artbase.rhizome.org/proxy/wdqs/bigdata/namespace/wdq/sparql"

#query to get all artworks and available information 
query_all_artworks = """SELECT ?artwork_page ?artwork_label ?artist_label ?accession ?inception ?summary_url ?description_url ?statement_url
{ 
  ?artwork rdfs:label ?artwork_label ;
   rt:P3 r:Q5 ;
   rt:P29 ?artist .
   ?artist rdfs:label ?artist_label . 
   ?artwork_page schema:about ?artwork .
  OPTIONAL {
   ?artwork rt:P85 ?accession ;
           rt:P26 ?inception . }
  OPTIONAL {
  ?artwork rt:P123 ?summary_description .
  ?summary_description rt:P3 r:Q4985 ;
          rdfs:label ?summary_label .
  ?summary_url schema:about ?summary_description .}
  OPTIONAL {
  ?artwork rt:P123 ?description .
  ?description rt:P3 r:Q9759 ;
          rdfs:label ?description_label .
  ?description_url schema:about ?description . }
  OPTIONAL {
    ?artwork rt:P123 ?statement .
  ?statement rt:P3 r:Q11838 ;
          rdfs:label ?statement_label .
  ?statement_url schema:about ?statement .
}}

"""
#collectives and their individual members
query_artists = '''SELECT ?subject ?member ?collectiveLabel ?memberLabel
WHERE { 
  ?subject rt:P3 r:Q7;
           rdfs:label ?collectiveLabel;
           rt:P43 ?member.
  ?member rdfs:label ?memberLabel.
  
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }}'''

#artists who aren't in a collective 
query_collective_filter = '''SELECT DISTINCT ?artistLabel ?artistPage
WHERE {
?artwork rt:P3 r:Q5.
?artwork rt:P29 ?artist.
?artist rt:P3 r:Q6.
?artistPage schema:about ?artist ;
schema:isPartOf <https://artbase.rhizome.org/> .
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
FILTER (?artist not in (r:Q7) )
}
ORDER BY ?artistLabel
}'''

results = get_results(endpoint_url, query_all_artworks)

for result in results["results"]["bindings"]:
    print(result)

The resulting datasets were however not complete enough for us to conduct a meaningful analysis. The major issue was that very few of Rhizome's artists had biographical information in the ArtBase, and with gender and nationality being key variables for our exploration we needed to complete this information. We tried a variety of automated ways -- including using Google's [Graph Search API](https://developers.google.com/knowledge-graph) -- however they proved limiting as the majority of artists in Rhizome's ArtBase are not popular enough to appear in Google's graph or if they did they were likely to have a name similar to someone else. <br><br>
In the end we decided to complete our dataset manually by conducting searches for nationality, gender, birth and death dates as well as WikiData and ULAN IDs. We used a simple common procedure: take the artist name and input it into Google and [Getty's ULAN database](https://www.getty.edu/research/tools/vocabularies/ulan/); from there try to quickly assert if information is available and input it. We also gave ourselves the following leeway: if an artist's gender or nationality was not clearly stated we inferred it from available text (such as the use of pronouns and primary places of work/study), and we added this into a separate gender/nationality column marked as guessed to differentiate it from the confirmed information. In total we guessed/inferred 224 nationalities and 233 genders.<br><br>
The final results of this work can be seen in the artists_complete.csv file held in the Rhizome_data folder. <br><br>
Following this we ran the Rhizome datasets through some scripts to align them with the MoMA datasets as well as perform the same linking exercise using an ID system that would allow us to have an artwork dataset containing gender and nationality information for our analysis. At this point guessed/inferred gender and nationality were combined with the confirmed information. We also decided that artists who were part of a collective would be treated as a collective only unless they also appeared in the ArtBase as a solo artist or unless the collective was larger than 2 artists. This decision was made to allow us to increase the Rhizome dataset as much as possible to help with our analysis, and was refined over various iterations including some discoveries during the scraping stage (see below) which revealed that Rhizome contained artworks on their site that were not yet fully catalogued in the ArtBase. <br><br>
As a result the total artists dataset increased from 1,272 entries to 1,299, while the artworks dataset remained constant at 2,270 (Rhizome publicly claims the ArtBase to contain more than 2,200 artworks). 


In [ ]:
#create a df for artists, pickle 
rhz_artists= pd.read_csv('./Rhizome_data/csv/artists_complete.csv', dtype='string')
#set empty fields w/ missing
rhz_artists['artistLabel'] = rhz_artists['artistLabel'].fillna('')
rhz_artists['artistPage'] = rhz_artists['artistPage'].fillna('')
rhz_artists['Nationality'] = rhz_artists['Nationality'].fillna('')
rhz_artists['Nationality Guessed'] = rhz_artists['Nationality Guessed'].fillna('')
rhz_artists['Gender'] = rhz_artists['Gender'].fillna('')
rhz_artists['Gender Guessed'] = rhz_artists['Gender Guessed'].fillna('')
rhz_artists['Birth'] = rhz_artists['Birth'].fillna('0').astype('int')
rhz_artists['Death'] = rhz_artists['Death'].fillna('0').astype('int')
rhz_artists['Wiki QID'] = rhz_artists['Wiki QID'].fillna('')
rhz_artists['ULAN'] = rhz_artists['ULAN'].fillna('')
#set collective fields to empty not missing 
rhz_artists['collectiveLabel'] = rhz_artists['collectiveLabel'].fillna('')
rhz_artists['collectivePage'] = rhz_artists['collectivePage'].fillna('')
#create and populate ID field to cross reference with artworks 
rhz_artists['ID'] = range(1, 1+len(rhz_artists))
rhz_artists['ID'] = rhz_artists['ID'].astype('string')
#drop the notes column 
rhz_artists = rhz_artists.drop('Notes', axis=1)
#rename columns 
rhz_artists.rename(columns={'artistLabel': 'Artist', 'artistPage': 'Artist URL', 'collectiveLabel': 'Collective', 'collectivePage': 'Collective URL', 'Wiki QID': 'WikiQID', 'Nationality Guessed': 'nationalityGuessed', 'Gender Guessed': 'genderGuessed', 'Wiki QID': 'wikiQID'}, inplace=True)

#rhz_artists.to_pickle(path+'Rhizome_data/pickles/rhizome_artists.pkl')

#create a second artists DF w/ concataned gender and nationality columns 
rhz_artists_extra = rhz_artists.copy()
rhz_artists_extra['Gender'] = rhz_artists_extra['Gender'].str.cat(rhz_artists_extra['genderGuessed'], join='outer', na_rep='missing')
rhz_artists_extra = rhz_artists_extra.drop('genderGuessed', axis=1)
rhz_artists_extra.loc[rhz_artists_extra['Gender'] == '', 'Gender'] = 'missing'
rhz_artists_extra['Nationality'] = rhz_artists_extra['Nationality'].str.cat(rhz_artists_extra['nationalityGuessed'], join='outer', na_rep='missing')
rhz_artists_extra = rhz_artists_extra.drop('nationalityGuessed', axis=1)
rhz_artists_extra.loc[rhz_artists_extra['Nationality'] == '', 'Nationality'] = 'missing'

#rhz_artists_extra.to_csv(path+'Rhizome_data/pickles/rhizome_artists_extra.csv')
#rhz_artists_extra.to_pickle(path+'Rhizome_data/pickles/rhizome_artists_extra.pkl')

#create a df for artworks, pickle 
rhz_artworks= pd.read_csv('./Rhizome_data/csv/artwork_complete.csv', dtype='string')
#set empty fields w/ missing or empty strings 
rhz_artworks['accession'] = rhz_artworks['accession'].fillna('0')
rhz_artworks.loc[:,'accession'] = rhz_artworks['accession'].where((rhz_artworks['accession'].str.len() <= 4), rhz_artworks['accession'].str[0:4])
rhz_artworks['inception'] = rhz_artworks['inception'].fillna('0')
rhz_artworks.loc[:,'inception'] = rhz_artworks['inception'].where((rhz_artworks['inception'].str.len() <= 4), rhz_artworks['inception'].str[0:4])
url_columns = ['summary_url', 'summary_url_2', 'description_url', 'description_url_2', 'statement_url']
rhz_artworks[url_columns] = rhz_artworks[url_columns].fillna('')
#rename columns 
rhz_artworks.rename(columns={'artwork_label': 'Title', 'artist_label': 'Artist', 'accession': 'dateAcquired', 'inception': 'dateCreated', 'artwork_page': 'URL'}, inplace=True)
#auto populate ID based on artist df 
for index, row in rhz_artworks.iterrows():
    ids = list()
    artist = getattr(row, 'Artist')
    artist = artist.split(', ')
    artists = rhz_artists_extra[['Artist', 'ID']]
    artists = dict(list(zip(artists.Artist, artists.ID)))
    collective_ids = defaultdict(list)
    for i, j in zip(rhz_artists_extra.Collective,rhz_artists_extra.ID):
        collective_ids[i].append(j)
    collective_ids = dict(collective_ids)
    del collective_ids['']
    for k, v in collective_ids.items():
        collective_ids.update({k: ', '.join([n for n in v])})

    for person in artist:
        if person in artists:
            ids.append(str(artists[person]))
        elif person in collective_ids:
            ids.append(collective_ids[person])
        else:
            ids.append('missing')
    string = ', '.join(ids)

    rhz_artworks.at[index, 'ID'] = string

#rhz_artworks.to_pickle(path+'Rhizome_data/pickles/rhizome_artworks.pkl')

#create a second df for artworks w/ additional artist info for visualisation 
rhz_artworks_extra = rhz_artworks.copy()
rhz_artworks_extra.drop(['summary_url', 'summary_url_2', 'description_url', 'description_url_2', 'statement_url'], inplace=True, axis=1)

#add nationality 
for index, row in rhz_artworks_extra.iterrows():
    nationality = list()
    artist = getattr(row, 'Artist')
    artist = artist.split(', ')
    artists = rhz_artists_extra[['Artist', 'Nationality']]
    artists = dict(list(zip(artists.Artist, artists.Nationality)))
    collectives_nat = defaultdict(list)
    for i, j in zip(rhz_artists_extra.Collective,rhz_artists_extra.Nationality):
        collectives_nat[i].append(j)
    collectives_nat = dict(collectives_nat)
    del collectives_nat['']
    for k, v in collectives_nat.items():
        collectives_nat.update({k: ', '.join([n for n in v])})

    for person in artist:
        if person in artists:
            nationality.append(str(artists[person]))
        elif person in collectives_nat:
            nationality.append(collectives_nat[person])
        else:
            nationality.append('missing')
    string_nationality = ', '.join(nationality)

    rhz_artworks_extra.at[index, 'Nationality'] = string_nationality
    
#add gender 
for index, row in rhz_artworks_extra.iterrows():
    gender = list()
    artist = getattr(row, 'Artist')
    artist = artist.split(', ')
    artists = rhz_artists_extra[['Artist', 'Gender']]
    artists = dict(list(zip(artists.Artist, artists.Gender)))
    collectives_gen = defaultdict(list)
    for i, j in zip(rhz_artists_extra.Collective,rhz_artists_extra.Gender):
        collectives_gen[i].append(j)
    collectives_gen = dict(collectives_gen)
    del collectives_gen['']
    for k, v in collectives_gen.items():
        collectives_gen.update({k: ', '.join([n for n in v])})

    for person in artist:
        if person in artists:
            gender.append(str(artists[person]))
        elif person in collectives_gen:
            gender.append(collectives_gen[person])
        else:
            gender.append('missing')
    string_gender = ', '.join(gender)

    rhz_artworks_extra.at[index, 'Gender'] = string_gender

#rhz_artworks_extra.to_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra.pkl')
#rhz_artworks_extra.to_csv(path+'Rhizome_data/pickles/rhizome_artworks_extra.csv')

In [12]:
#show result
rhz_artworks_extra = pd.read_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra.pkl')
rhz_artworks_extra.head()

,ID,URL,Title,Artist,dateAcquired,dateCreated,Nationality,Gender
0,879,https://artbase.rhizome.org/wiki/Q2423,ZUR FARBENLEHRE (THEORY OF COLOURS),Steven Jones,2007,2007,British,M
1,1020,https://artbase.rhizome.org/wiki/Q4089,Zones de Convergence,cicero,2005,2005,missing,missing
2,"243, 701",https://artbase.rhizome.org/wiki/Q1475,Zombie and Mummy,"Dragan Espenschied, Olia Lialina",2004,2002,"German, Russian","M, F"
3,312,https://artbase.rhizome.org/wiki/Q4374,"Zaira, City of Memories",Gokcen Erguven,2004,2004,Turkish,F
4,920,https://artbase.rhizome.org/wiki/Q3972,Z_G [zeitgeist gestalten],Tiago Borges,2008,2007,Angolan,M


***

## Scraping Text (Laurent)

One last data preparation step involved scraping text about artworks from both MoMA and Rhizome's websites. This was suggested early on in our exploration process and as we came to better understand the differences between the available datasets we decided that grabbing the text might prove useful as an additional variable for our analysis. In addition it was decided that by doing this we might be able to create department/medium information for Rhizome somewhat equivalent to MoMA's.<br><br>
After some trials with the html construction of the Rhizome artwork pages we decided to grab the `<div>` that includes all three possible descriptions (summary, artist statement, description).  

In [ ]:
#COMMENTED OUT TO AVOID BINDER RUNNING IT - grab URLs from artworks DF, scrape them and return them back to the DF
#rhz_artworks_extra = pd.read_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra.pkl')
#urls = rhz_artworks_extra['URL'].to_list()
#scrapes = [url_to_text_rhizome(u) for u in urls]
#rhz_artworks_extra_text = rhz_artworks_extra.copy()
#rhz_artworks_extra_text['Text'] = pd.Series(scrapes)
#fix an erroneous ID in original first round of scraping 
#rhz_artworks_extra_text.loc[777, 'ID'] = '926, 1268'
#rhz_artworks_extra_text = rhz_artworks_extra_text.astype(str)
#rhz_artworks_extra_text.to_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra_text.pkl')

In [14]:
#show result
rhz_artworks_extra_text = pd.read_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra_text.pkl')
rhz_artworks_extra_text.head()

,ID,URL,Title,Artist,dateAcquired,dateCreated,Nationality,Gender,Text
0,879,https://artbase.rhizome.org/wiki/Q2423,ZUR FARBENLEHRE (THEORY OF COLOURS),Steven Jones,2007,2007,British,M,"['summary edit\n\n\t\t\t\tA short film.\n""When..."
1,1020,https://artbase.rhizome.org/wiki/Q4089,Zones de Convergence,cicero,2005,2005,missing,missing,['description edit\n\n\t\t\t\t‘Zones de Conver...
2,"243, 701",https://artbase.rhizome.org/wiki/Q1475,Zombie and Mummy,"Dragan Espenschied, Olia Lialina",2004,2002,"German, Russian","M, F",['summary edit\n\n Zombie and Mummy is a websi...
3,312,https://artbase.rhizome.org/wiki/Q4374,"Zaira, City of Memories",Gokcen Erguven,2004,2004,Turkish,F,['description edit\n\n\t\t\t\tThe project is b...
4,920,https://artbase.rhizome.org/wiki/Q3972,Z_G [zeitgeist gestalten],Tiago Borges,2008,2007,Angolan,M,['description edit\n\n\t\t\t\tfrom looking at ...


For MOMA URLs there was only one possible description on the page but the containing `<div>` is repeated elsewhere so we used its parent container to only extract what we needed. Considering the size of the MoMA dataset we used the department splits created earlier to go through it in chunks. We show the process for one dept only, but this was repeated for all of them.

In [ ]:
#load department DFs
moma_arch_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont.pkl')
moma_arch_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod.pkl')
moma_design_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_img_cont.pkl')
moma_design_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_img_mod.pkl')
moma_draw_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont.pkl')
moma_draw_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod.pkl')
moma_films_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/films_cont.pkl')
moma_films_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/films_mod.pkl')
moma_fluxus_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_cont.pkl')
moma_fluxus_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/fluxus_mod.pkl')
moma_media_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_cont.pkl')
moma_media_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_mod.pkl')
moma_paint_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont.pkl')
moma_paint_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod.pkl')
moma_photo_cont = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_cont.pkl')
moma_photo_mod = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_mod.pkl')

#COMMENTED OUT TO AVOID BINDER RUNNING IT - grab all links from a dept as a list 
#links = moma_photo_mod['URL'].to_list()
#process links w/ function 
#moma_photo_to_add = [url_to_text_moma(u) for u in links]
#add results back to a copy of the original DF
#moma_photo_mod_text = moma_photo_mod.copy()
#moma_photo_mod_text['Text'] = moma_photo_to_add
#moma_photo_mod_text['Text'] = moma_photo_mod_text['Text'].astype(str)
#moma_photo_mod_text.to_pickle(path+'MOMA_data/pickle/departments/photo_mod_text.pkl')


In [16]:
#show results
moma_arch_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text.pkl')
moma_arch_cont_text.head()

,Title,Artist,ID,DateCreated,Medium,Department,DateAcquired,URL,ThumbnailURL,Nationality,Gender,Text
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,1987,Paint and colored pencil on print,Architecture & Design,1995,http://www.moma.org/collection/works/3,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,French,M,missing
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Photographic reproduction with colored synthet...,Architecture & Design,1995,http://www.moma.org/collection/works/5,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,missing,M,The Manhattan Transcripts are theoretical prop...
31,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Photographic reproduction with colored synthet...,Architecture & Design,1995,http://www.moma.org/collection/works/33,http://www.moma.org/media/W1siZiIsIjIwMCJdLFsi...,missing,M,The Manhattan Transcripts are theoretical prop...
35,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Photographic reproduction with colored synthet...,Architecture & Design,1995,http://www.moma.org/collection/works/38,http://www.moma.org/media/W1siZiIsIjI2NyJdLFsi...,missing,M,The Manhattan Transcripts are theoretical prop...
40,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Ink on tracing paper,Architecture & Design,1995,http://www.moma.org/collection/works/44,http://www.moma.org/media/W1siZiIsIjI5NiJdLFsi...,missing,M,The Manhattan Transcripts are theoretical prop...


### Cleaning & Keywords

Scraping rhizome resulted in text that heavy with encoding noise. We cleaned it as best we could without damaging the underlying text and then analyzed the text for the most common words usng NLTK's stopwords and our own custom list based on analysis on the scraped results and some early tests. We then extracted 20 keywords from each text and assigned those to their own column. 

In [ ]:
#load the DF w/ scraped text
rhz_artworks_extra_text = pd.read_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra_text.pkl')
rhz_artworks_extra_text = rhz_artworks_extra_text.astype('string')

#remove all punctuation
rhz_artworks_extra_text['Text'] = rhz_artworks_extra_text['Text'].replace(regex=r'[^\w\s]', value='')

#create a list of strings to remove from all scrapes
remove = ['description editnntttt', '[', ']', 'nnn', 'description edit', 'editnn', 'summary edit', 'tttt', 'nn']
for char in remove:
    rhz_artworks_extra_text['Text'] = rhz_artworks_extra_text['Text'].str.replace(char, '')

#export cleaned version to pickle
rhz_artworks_extra_text_clean = rhz_artworks_extra_text.copy()
rhz_artworks_extra_text_clean.to_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra_text_clean.pkl')

# create a list of stop words and add custom stopwords
stop_words = set(stopwords.words("english"))
new_words =  ['rhizome', 'attributed', 'summary', 'inception', 'staff', 'attribution', 'summary', 'legacy', 'nattributed']
stop_words = stop_words.union(new_words)

#remove stopwords from cleanedtext 
rhz_artworks_extra_text_clean['Text'] = rhz_artworks_extra_text_clean['Text'].str.lower()
rhz_artworks_extra_text_clean['Text'] = rhz_artworks_extra_text_clean['Text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))
#remove errand string
rhz_artworks_extra_text_clean['Text'] = rhz_artworks_extra_text_clean['Text'].replace(regex={r'.attributed.': '', r'.legacy descriptive tags.': '', r'.attribution: staff.': '', r'.attribution.': '', r'.inception.': '', r'http[a-z]*\s': ' '})

#extract kws and assign them to new column
rhz_artworks_extra_text_clean['Keywords'] = rhz_artworks_extra_text_clean.apply(lambda row:get_keywords(row), axis=1) 

#export new pickle for reuse
rhz_artworks_extra_text_clean.to_pickle(path+'Rhizome_data/rhizome_artworks_extra_text_clean_stop_keywords.pkl')


In [18]:
#show results
rhz_artworks_extra_text_clean = pd.read_pickle(path+'Rhizome_data/pickles/rhizome_artworks_extra_text_clean_stop_keywords.pkl')
rhz_artworks_extra_text_clean.head()

,ID,URL,Title,Artist,dateAcquired,dateCreated,Nationality,Gender,Text,Keywords
0,879,https://artbase.rhizome.org/wiki/Q2423,ZUR FARBENLEHRE (THEORY OF COLOURS),Steven Jones,2007,2007,British,M,short filmnwhen eye sees colour immediately ex...,"colour, march, eye, sees, immediately, excited..."
1,1020,https://artbase.rhizome.org/wiki/Q4089,Zones de Convergence,cicero,2005,2005,missing,missing,zones de convergence treats days summit g8 evi...,"summit, art, august, zones, de, convergence, t..."
2,"243, 701",https://artbase.rhizome.org/wiki/Q1475,Zombie and Mummy,"Dragan Espenschied, Olia Lialina",2004,2002,"German, Russian","M, F",zombie mummy website hosted episodic online co...,"zombie, site, olia, espenschied, mummy, advent..."
3,312,https://artbase.rhizome.org/wiki/Q4374,"Zaira, City of Memories",Gokcen Erguven,2004,2004,Turkish,F,project based solely italo calvinos novel call...,"cities, content, city, zaira, project, based, ..."
4,920,https://artbase.rhizome.org/wiki/Q3972,Z_G [zeitgeist gestalten],Tiago Borges,2008,2007,Angolan,M,looking den zeitgeist gestalten design spirit ...,"experience, web, page, den, zeitgeist, gestalt..."


For MoMA the scrapes were cleaner of encoding noise so we simply ran everything through the same stopwords (and a different set of custom stopwords) and then removed empty entries. Out of over 138K artworks in MoMA's dataset we found text for 2,880 items.<br><br>
**NB**: while scraping one of the larger departments we made an error that resulted in the dataframe including only the extracted text (with the other original information dropped). Rather than rescrape everything we kept this DF but subjected it to slightly different processing as seen below.

In [ ]:
#load DFs with text
moma_arch_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text.pkl')
moma_arch_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only.pkl')
moma_draw_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text.pkl')
moma_draw_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text.pkl')
moma_films_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/films_cont_text.pkl')
moma_films_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/films_mod_text.pkl')
moma_media_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text.pkl')
moma_media_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text.pkl')
moma_paint_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text.pkl')
moma_paint_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text.pkl')
moma_photo_cont_text = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_cont_text.pkl')
moma_photo_mod_text = pd.read_pickle(path+'MOMA_data/pickle/departments/photo_mod_text.pkl')

#DFs to list then clean them 
moma_texts = [moma_arch_cont_text, moma_draw_cont_text, moma_draw_mod_text, moma_films_cont_text, moma_films_mod_text, moma_media_cont_text, moma_media_mod_text, moma_paint_cont_text, moma_paint_mod_text, moma_photo_cont_text, moma_photo_mod_text]

remove = ['[', ']', '</p>', '<p>', '<strong>', '</strong>', '<em>', '</em>', '</br>']
for text in moma_texts:
    for char in remove:
        text['Text'] = text['Text'].str.replace(char, '')

#exception for DF that was formatted differently due to scraping error (it has one column only w/ scraped text)
for char in remove:
    moma_arch_mod_text['text'] = moma_arch_mod_text['text'].str.replace(char, '')

#replace empty strings w/ missing value to filter useful results only
moma_arch_cont_text['Text'] = moma_arch_cont_text['Text'].replace([''], 'missing')
moma_arch_mod_text['text'] = moma_arch_mod_text['text'].replace([''], 'missing')
moma_draw_cont_text['Text'] = moma_draw_cont_text['Text'].replace([''], 'missing')
moma_draw_mod_text['Text'] = moma_draw_mod_text['Text'].replace([''], 'missing')
moma_films_cont_text['Text'] = moma_films_cont_text['Text'].replace([''], 'missing')
moma_films_mod_text['Text'] = moma_films_mod_text['Text'].replace([''], 'missing')
moma_media_cont_text['Text'] = moma_media_cont_text['Text'].replace([''], 'missing')
moma_media_mod_text['Text'] = moma_media_mod_text['Text'].replace([''], 'missing')
moma_paint_cont_text['Text'] = moma_paint_cont_text['Text'].replace([''], 'missing')
moma_paint_mod_text['Text'] = moma_paint_mod_text['Text'].replace([''], 'missing')
moma_photo_cont_text['Text'] = moma_photo_cont_text['Text'].replace([''], 'missing')
moma_photo_mod_text['Text'] = moma_photo_mod_text['Text'].replace([''], 'missing')

#save cleaned DFs back 
moma_arch_cont_text.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text.pkl')
moma_arch_mod_text.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only.pkl')
moma_draw_cont_text.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text.pkl')
moma_draw_mod_text.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text.pkl')
moma_films_cont_text.to_pickle(path+'MOMA_data/pickle/departments/films_cont_text.pkl')
moma_films_mod_text.to_pickle(path+'MOMA_data/pickle/departments/films_mod_text.pkl')
moma_media_cont_text.to_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text.pkl')
moma_media_mod_text.to_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text.pkl')
moma_paint_cont_text.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text.pkl')
moma_paint_mod_text.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text.pkl')
moma_photo_cont_text.to_pickle(path+'MOMA_data/pickle/departments/photo_cont_text.pkl')
moma_photo_mod_text.to_pickle(path+'MOMA_data/pickle/departments/photo_mod_text.pkl')

#reduce DFs down to entries w/ only useful text and remove duplicates 
moma_arch_cont_text_final = moma_arch_cont_text.loc[moma_arch_cont_text['Text'] != 'missing']
moma_arch_cont_text_final = moma_arch_cont_text_final.drop_duplicates(subset=['Text'])
moma_arch_mod_text_final = moma_arch_mod_text.loc[moma_arch_mod_text['text'] != 'missing']
moma_arch_mod_text_final = moma_arch_mod_text_final.drop_duplicates(subset=['text'])
moma_draw_cont_text_final = moma_draw_cont_text.loc[moma_draw_cont_text['Text'] != 'missing']
moma_draw_cont_text_final = moma_draw_cont_text_final.drop_duplicates(subset=['Text'])
moma_draw_mod_text_final = moma_draw_mod_text.loc[moma_draw_mod_text['Text'] != 'missing']
moma_draw_mod_text_final = moma_draw_mod_text_final.drop_duplicates(subset=['Text'])
moma_films_cont_text_final = moma_films_cont_text.loc[moma_films_cont_text['Text'] != 'missing']
moma_films_cont_text_final = moma_films_cont_text_final.drop_duplicates(subset=['Text'])
moma_films_mod_text_final = moma_films_mod_text.loc[moma_films_mod_text['Text'] != 'missing']
moma_films_mod_text_final = moma_films_mod_text_final.drop_duplicates(subset=['Text'])
moma_media_cont_text_final = moma_media_cont_text.loc[moma_media_cont_text['Text'] != 'missing']
moma_media_cont_text_final = moma_media_cont_text_final.drop_duplicates(subset=['Text'])
moma_media_mod_text_final = moma_media_mod_text.loc[moma_media_mod_text['Text'] != 'missing']
moma_media_mod_text_final = moma_media_mod_text_final.drop_duplicates(subset=['Text'])
moma_paint_cont_text_final = moma_paint_cont_text.loc[moma_paint_cont_text['Text'] != 'missing']
moma_paint_cont_text_final = moma_paint_cont_text_final.drop_duplicates(subset=['Text'])
moma_paint_mod_text_final = moma_paint_mod_text.loc[moma_paint_mod_text['Text'] != 'missing']
moma_paint_mod_text_final = moma_paint_mod_text_final.drop_duplicates(subset=['Text'])
moma_photo_cont_text_final = moma_photo_cont_text.loc[moma_photo_cont_text['Text'] != 'missing']
moma_photo_cont_text_final = moma_photo_cont_text_final.drop_duplicates(subset=['Text'])
moma_photo_mod_text_final = moma_photo_mod_text.loc[moma_photo_mod_text['Text'] != 'missing']
moma_photo_mod_text_final = moma_photo_mod_text_final.drop_duplicates(subset=['Text'])

#save DFs
moma_arch_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text_final.pkl')
moma_arch_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only_final.pkl')
moma_draw_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text_final.pkl')
moma_draw_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text_final.pkl')
moma_films_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/films_cont_text_final.pkl')
moma_films_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/films_mod_text_final.pkl')
moma_media_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text_final.pkl')
moma_media_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text_final.pkl')
moma_paint_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text_final.pkl')
moma_paint_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text_final.pkl')
moma_photo_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/photo_cont_text_final.pkl')
moma_photo_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/photo_mod_text_final.pkl')

#load stopwords and remove them
stop_words = set(stopwords.words("english"))
new_words =  ['.', 'one', 'two', 'also']
stop_words = stop_words.union(new_words)

moma_texts_final = [moma_arch_cont_text_final, moma_draw_cont_text_final, moma_draw_mod_text_final, moma_films_cont_text_final, moma_films_mod_text_final, moma_media_cont_text_final, moma_media_mod_text_final, moma_paint_cont_text_final, moma_paint_mod_text_final, moma_photo_cont_text_final, moma_photo_mod_text_final]

for text in moma_texts_final:
    text['Text'] = text['Text'].str.lower()
    text['Text'] = text['Text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))

#exception 
moma_arch_mod_text_final['text'] = moma_arch_mod_text_final['text'].str.lower()
moma_arch_mod_text_final['text'] = moma_arch_mod_text_final['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop_words]))

#get keywords
moma_arch_cont_text_final['Keywords'] = moma_arch_cont_text_final.apply(lambda row:get_keywords(row), axis=1)
moma_draw_cont_text_final['Keywords'] = moma_draw_cont_text_final.apply(lambda row:get_keywords(row), axis=1) 
moma_draw_mod_text_final['Keywords'] = moma_draw_mod_text_final.apply(lambda row:get_keywords(row), axis=1) 
moma_films_cont_text_final['Keywords'] = moma_films_cont_text_final.apply(lambda row:get_keywords(row), axis=1)
moma_films_mod_text_final['Keywords'] = moma_films_mod_text_final.apply(lambda row:get_keywords(row), axis=1) 
moma_media_cont_text_final['Keywords'] = moma_media_cont_text_final.apply(lambda row:get_keywords(row), axis=1)
moma_media_mod_text_final['Keywords'] = moma_media_mod_text_final.apply(lambda row:get_keywords(row), axis=1)
moma_paint_cont_text_final['Keywords'] = moma_paint_cont_text_final.apply(lambda row:get_keywords(row), axis=1)
moma_paint_mod_text_final['Keywords'] = moma_paint_mod_text_final.apply(lambda row:get_keywords(row), axis=1)
moma_photo_cont_text_final['Keywords'] = moma_photo_cont_text_final.apply(lambda row:get_keywords(row), axis=1)
moma_photo_mod_text_final['Keywords'] = moma_photo_mod_text_final.apply(lambda row:get_keywords(row), axis=1)
moma_arch_mod_text_final['Keywords'] = moma_arch_mod_text_final.apply(lambda row:get_keywords_2(row), axis=1) 

#pickle version w/ stopwords removed and keywords added
moma_arch_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text_final_stop.pkl')
moma_arch_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/architecture_design_mod_text_only_final_stop.pkl')
moma_draw_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_cont_text_final_stop.pkl')
moma_draw_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/draws_prints_mod_text_final_stop.pkl')
moma_films_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/films_cont_text_final_stop.pkl')
moma_films_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/films_mod_text_final_stop.pkl')
moma_media_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/media_perf_cont_text_final_stop.pkl')
moma_media_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/media_perf_mod_text_final_stop.pkl')
moma_paint_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_cont_text_final_stop.pkl')
moma_paint_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/paint_sculp_mod_text_final_stop.pkl')
moma_photo_cont_text_final.to_pickle(path+'MOMA_data/pickle/departments/photo_cont_text_final_stop.pkl')
moma_photo_mod_text_final.to_pickle(path+'MOMA_data/pickle/departments/photo_mod_text_final_stop.pkl')


In [19]:
#show results
moma_arch_cont_text_final = pd.read_pickle(path+'MOMA_data/pickle/departments/architecture_design_cont_text_final_stop.pkl')
moma_arch_cont_text_final.head()

,Title,Artist,ID,DateCreated,Medium,Department,DateAcquired,URL,ThumbnailURL,Nationality,Gender,Text,Keywords
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Photographic reproduction with colored synthet...,Architecture & Design,1995,http://www.moma.org/collection/works/5,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,missing,M,manhattan transcripts theoretical propositions...,"events, architecture, manhattan, transcripts, ..."
35,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,1980,Photographic reproduction with colored synthet...,Architecture & Design,1995,http://www.moma.org/collection/works/38,http://www.moma.org/media/W1siZiIsIjI2NyJdLFsi...,missing,M,manhattan transcripts theoretical propositions...,"events, architecture, manhattan, movement, tra..."
151,"Slow House Project, North Haven, New York, Pla...","Diller + Scofidio, Elizabeth Diller, Ricardo S...","8707, 6951, 6952",1989,Computer-generated print on frosted polymer sh...,Architecture & Design,1992,http://www.moma.org/collection/works/201,http://www.moma.org/media/W1siZiIsIjE5MjQiXSxb...,"American, American, American","missing, F, M",elizabeth diller ricardo scofidio's slow house...,"house, drawing, view, diller, scofidio, city, ..."
152,"The Peak Project, Hong Kong, China (Exterior p...",Zaha Hadid,6953,1991,Synthetic polymer on paper mounted on canvas,Architecture & Design,1992,http://www.moma.org/collection/works/202,http://www.moma.org/media/W1siZiIsIjE5MzgiXSxb...,British,F,painting depicts hadid’s winning entry archite...,"hadid, painting, hills, structure, forms, winn..."
170,"Chikatsu-Asuka Historical Museum, Minami-Kawac...",Tadao Ando,7055,1989,Graphite and crayon with scoring on paper,Architecture & Design,1993,http://www.moma.org/collection/works/224,http://www.moma.org/media/W1siZiIsIjI0MjEiXSxb...,Japanese,M,tadao ando's drawings like hieroglyphic map re...,"ando, museum, building, landscape, tadao, like..."


***

## Manipulating and Visualising the Data (Everyone)

For our manipulation and visualisation of the data we decided on a workflow inspired by what we learnt during the class. Starting from our initial research question -- Did the internet and personal computers have an impact on the make up of art collections? -- we looked at the types of variable we had available, their potential relationships, and the ways in which this could help us explore and visualize the data. <br><br>
We ended up deciding on the following variables as essential to our exploration: 
- Gender (categorical/nominal)
- Nationality (categorical/nominal)
- Date Created and Acquired (ordinal)
- Departments (categorical)
- Keywords (treated as numerical)

Considering the difference in size between our datasets we eventually decided to settle on the following approach to help our manipulation, visualization, and storytelling: the Rhizome dataset would be used in full, MoMA would only be used in full untouched for the first visualization and subsequently would only be used as either full with the 1983 pivot date (to see if the internet and personal computers might have made a difference in acquisitions) or as a trimmed-down version for comparison with Rhizome, aligned to Rhizome by using the same creation date for the first artwork (1893) and the same first acquisition date (2000).<br><br> 
Therefore we would in effect have three versions of the MoMA dataset: the full version (138K items); a version split at 1983 as a creation date (107,339 items vs 30,812); and a version trimmed alongside Rhizome's key dates (21,580 items). With this we felt we could more confidently work on the data as when comparing MoMA before and after 1983 we could divide any needed counts for the pre-1983 period by 30% and when comparing MoMA and Rhizome we could divide needed counts by 10%.<br><br>
From there we discussed what types of visualizations we might want -- comparisons, relationships, distribution, composition etc... -- and ended up with a list of questions expanding from the initial research question and which became a guide for both creating visualizations and building the story. For the latter we also made use of Miro to create a storyboard of the website and help decide which sub question/visualization would prove essential. <br><br>
The final list of questions and their associated viusalizations and datasets were:
1. **What is the gender representation in each dataset?**
    - Rhizome vs MoMA full + MoMA before and after 1983 
    - Pie charts -> gender as categorical and numerical  
2. **How does gender representation compare between both datasets?** 
    - Rhizome vs MoMA sampled
    - Pie charts -> gender as categorical and numerical 
3. **How many artworks were created by collectives and what is their gender breakdown?** 
    - Rhizome vs MoMA sampled
    - Sunburst charts -> gender as categorical and numerical  
4. **How does gender representation compare between datasets alongside departments?** 
    - Rhizome vs MoMA sampled + MoMA before and after 1983 
    - Barcharts -> gender as categorical and numerical + departments as categorical
5. **What is the nationality representation in each dataset?**
    - Rhizome vs MoMA sampled 
    - Map -> nationality as categorical and numerical  
6. **How does nationality representation compare between datasets?** 
    - Rhizome vs MoMA sampled
    - Bar charts -> nationality as numerical + dataset as categorical (with additional filtering using the concept of Global North/South divide)
7. **How does nationality relate to gender?** 
    - Rhizome vs MoMA sampled
    - Parallel chart -> gender as categorical + department as categorical + nationality as categorical and numerical (with some additional filtering using the concept of Global North/South divide)
8. **Which years have the highest acquisition date in relation to gender and/or nationality?**
    - Rhizome vs MoMA sampled 
    - Scatter plot -> date created/acquired as ordinal + gender/nationality as categorical and numerical (with some additional filtering using the concept of Global North/South divide)
9. **Which keywords are most prominent in artwork descriptions?** 
    - Rhizome vs MoMA TBC
    - Tree maps -> TBC
10. **How do keywords relate to gender and/or nationality?**
    - TBC
    - TBC

Let's do this! 💪

***

### Representing Gender (Margherita & Laurent)

Three sets of two pie charts and one set of two sunburst charts.

TO DO 
- Align legend/category order 
- Make a note in story that collectives include artists who worked together but may not be a collective as such because of inability to easily identify collectives in MoMA

In [60]:
#Rhizome vs MoMA full
rhz_gender_rep = group_genders(rhz_artworks)
moma_gender_rep = group_genders(moma_artworks)

rhz_gender_pie = px.pie(rhz_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of Rhizome Collection')
rhz_gender_pie.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
rhz_gender_pie.update_layout(width=800, title_x=0.5)

moma_gender_pie = px.pie(moma_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection (Full)')
moma_gender_pie.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
moma_gender_pie.update_layout(width=800, title_x=0.5)

rhz_gender_pie = go.FigureWidget(rhz_gender_pie)
moma_gender_pie = go.FigureWidget(moma_gender_pie)
gender_pies = ipw.HBox([rhz_gender_pie, moma_gender_pie])
gender_pies

    'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hover…

In [61]:
#Rhizome vs MoMA sampled
moma_rhz_compare_gender_rep = group_genders(moma_rhz_compare)

moma_gender_pie_2 = px.pie(moma_rhz_compare_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection (Sampled)')
moma_gender_pie_2.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
moma_gender_pie_2.update_layout(width=800, title_x=0.5)

moma_gender_pie_2 = go.FigureWidget(moma_gender_pie_2)
gender_pies_2 = ipw.HBox([rhz_gender_pie, moma_gender_pie_2])
gender_pies_2

    'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hover…

In [20]:
#create a DF w/ genders and counts, group collectives together
moma_b4_gender_rep = group_genders(moma_artworks_old)
moma_after_gender_rep = group_genders(moma_artworks_new)

moma_gender_pie_b4 = px.pie(moma_b4_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection Before 1983')
moma_gender_pie_b4.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
moma_gender_pie_b4.update_layout(width=800, title_x=0.5)

moma_gender_pie_after = px.pie(moma_after_gender_rep, values='Counts', names='Gender',
             title='Gender Breakdown of MoMA Collection After 1983')
moma_gender_pie_after.update_traces(textposition='auto', textinfo='label+percent', hoverinfo='label+value+percent', hovertemplate=None, pull=[0, 0, 0, 0.2])
moma_gender_pie_after.update_layout(width=800,  title_x=0.5)

moma_gender_pie_3 = go.FigureWidget(moma_gender_pie_b4)
moma_gender_pie_4 = go.FigureWidget(moma_gender_pie_after)
gender_pies_3 = ipw.HBox([moma_gender_pie_3, moma_gender_pie_4])
gender_pies_3

    'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hover…

In [66]:
moma_gender_collectives.to_csv(path+'moma_collectives.csv')

In [ ]:
#Rhizome vs MoMA sampled - collectives gender rep 

fig_sun =go.Figure(go.Sunburst(
    labels=["Collectives", "Males", "Male Only", "Male & Missing", "Females", "Female Only", "Female & Missing", "Mixed Gender", 'Including Missing Gender'],
    parents=["", "Collectives","Males", "Males", "Collectives", "Females", "Females", "Collectives", 'Mixed Gender'],
    values=[131, 37, 35, 2, 9, 6, 3, 85, 2]
),layout=dict(title=dict(text="Gender Breakdown of Rhizome Collectives")))
fig_sun.update_traces(textinfo='label+value', hoverinfo='label+value', hovertemplate=None)
fig_sun.update_layout(width=1000, height=1200)

fig_sun_2 =go.Figure(go.Sunburst(
    labels=["Collectives", "Males", "Male Only", "Male & Missing", "Females", "Female Only", "Female & Missing", "Mixed Gender", 'Including Missing Gender'],
    parents=["", "Collectives","Males", "Males", "Collectives", "Females", "Females", "Collectives", 'Mixed Gender'],
    values=[XX, 901, 435, 468, 206, 135, 71, 525, 131]
),layout=dict(title=dict(text="Gender Breakdown of MoMA Collectives")))
fig_sun_2.update_traces(textinfo='label+value', hoverinfo='label+value', hovertemplate=None)
fig_sun_2.update_layout(width=1000, height=1200)

fig_sun.show(), fig_sun_2.show()